<a href="https://colab.research.google.com/github/ForeverLMZ/Pharmahacks/blob/main/Pharmahacks_phyla_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import warnings
warnings.filterwarnings("ignore")

#Data Extraction

In [5]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('challenge_1_gut_microbiome_data').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(rows)

print(df)

df = df.iloc[1:]




             0           1           2           3           4           5     \
0                  Bacteria-1  Bacteria-2  Bacteria-3  Bacteria-4  Bacteria-5   
1        Sample-1           0           0           0           0           0   
2        Sample-2           0           0           0           0           0   
3        Sample-3           0          10           0           0           0   
4        Sample-4           0          17           0           0           0   
...           ...         ...         ...         ...         ...         ...   
7477  Sample-7477           0           0           0           0           0   
7478  Sample-7478           0          11           0           0           0   
7479  Sample-7479           0           0           0           0           0   
7480  Sample-7480           0           0           0           0           0   
7481  Sample-7481           0          14           0           0           0   

            6           7  

#Data Preprocessing & Normalization

Convert the labels to numerical values

0 - healthy

3 - disease 3

2 - disease 2

1 - disease 1

In [6]:
import numpy as np
#print(df[:][1095])
label = df[:][1095]
label = label.replace('Healthy',0)
label = label.replace('Disease-3',3)
label = label.replace('Disease-2',2)
label = label.replace('Disease-1',1)


label = label.to_numpy() #save the labels as a separate numpy array
y = label.astype(int) #convert the str into int

drops garbage label column and text columns

In [7]:
df = df.drop(df.columns[1095], axis=1)
df = df.drop(df.columns[0], axis=1)

In [8]:
x = df.to_numpy() #converts dataframe to numpy

normalize the dataset such that all data entries are normallized within [0,1] range, column-wise, using the min-max method.
(This deals with the feature imbalance problem)

In [9]:
print(len(df))
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x) #converts the input dataframe to numpy data structure
print(pd.DataFrame(x_scaled))
#print(x_scaled[0].max())

7481
      0         1     2     3     4     5     6     7     8     9     ...  \
0      0.0  0.000000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
1      0.0  0.000000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
2      0.0  0.000755   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
3      0.0  0.001283   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
4      0.0  0.000604   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
...    ...       ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
7476   0.0  0.000000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
7477   0.0  0.000830   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
7478   0.0  0.000000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
7479   0.0  0.000000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
7480   0.0  0.001056   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   

      1084  1085      1086  1087  1088  1089  1090  1091      1092  10

#Training Set & Validation Set Split

In [41]:
(N,D),C= x_scaled.shape,np.max(label)+1
print(N,D,C)

#we now splits 20% of the training set into validation set
inds = np.random.permutation(N)  # shuffle the data before splitting 
n_val = N//5
tr_inds = []
# get the validation indices
val_inds = list(range(0, n_val)) #get the validation indices
tr_inds = tr_inds + list(range(n_val, N)) #get the training indices

x_train = x_scaled[inds[tr_inds]]
y_train = y[inds[tr_inds]]
x_test = x_scaled[inds[val_inds]]
y_test = y[inds[val_inds]]


#we are good to go!

7481 1094 4


#Feature Selection

In [71]:
x_train_fs = x_train
x_test_fs = x_test

from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2
selector = SelectKBest(k=470) #selects the top 470 features (the k value hyperparameter-tuned by hand)
selector.fit(x_train, y_train)
x_train_fs = selector.transform(x_train)
x_test_fs = selector.transform(x_test)

#Model Implementation

In [12]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC

#Gaussian Naive Bayes  (NOT USED)

In [96]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(x_train, y_train)
GaussianNB()
print(clf.score(x_test,y_test))

0.32954545454545453


Cross-Validation Score

In [97]:
from sklearn.metrics import SCORERS
#print(sorted(SCORERS.keys()))
scores = cross_validate(clf, x_train, y_train, cv=5,return_train_score=False)

In [99]:
print(scores['test_score'])

[0.33751044 0.31578947 0.34085213 0.36591479 0.34670008]


#Gradient Boosting Classifier (NOT USED)

In [101]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(x_train, y_train)
print(clf.score(x_test, y_test))

0.7633689839572193


In [150]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(x_train_fs, y_train)
print(clf.score(x_test_fs, y_test))

0.7479946524064172


Cross-validation score

In [102]:
scores = cross_validate(clf, x_train, y_train, cv=5,return_train_score=False)
print(scores['test_score'])

[0.73517126 0.7443609  0.74686717 0.74686717 0.75438596]


#Extra Trees Classifier (USED!)

original performances without hyper-parameter tuning

In [42]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import make_classification
clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
clf.fit(x_train, y_train)
ExtraTreesClassifier(random_state=0)
print("Testing set score without feature selection")
print(clf.score(x_test,y_test))

Testing set score without feature selection
0.7834224598930482


In [72]:
clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
clf.fit(x_train_fs, y_train)
ExtraTreesClassifier(random_state=0)
print("Testing set score with feature selection")
print(clf.score(x_test_fs,y_test))

Testing set score with feature selection
0.7947860962566845


#cross-validation to test feature selection

In [73]:
#without feature selection
scores = cross_validate(clf, x_train, y_train, cv=5,return_train_score=False)
print(scores['test_score'])
#with feature selection
scores = cross_validate(clf, x_train_fs, y_train, cv=5,return_train_score=False)
print(scores['test_score'])

[0.80033417 0.786132   0.78529657 0.78696742 0.75689223]
[0.80284043 0.78780284 0.78696742 0.786132   0.75355054]


#Hyper-parameter Tuning of Extra Trees Classifier

In [74]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

clf = ExtraTreesClassifier()
params = [{'bootstrap': [False,True],'warm_start':[True,False]}] #the list of hyper-parameters to try
gs = GridSearchCV(clf,param_grid=params,scoring='f1',cv=3)
gs.fit(x_train_fs, y_train)


GridSearchCV(cv=3, estimator=ExtraTreesClassifier(),
             param_grid=[{'bootstrap': [False, True],
                          'warm_start': [True, False]}],
             scoring='f1')

In [75]:
print(gs.best_params_) #prints out the best performing hyper-parameter values

{'bootstrap': False, 'warm_start': True}


#Final Accuracy

In [83]:
clf = ExtraTreesClassifier(n_estimators=100, random_state=0,max_features = 'sqrt',warm_start=True) #rerun the model with all of the tuned hyper-parameters
clf.fit(x_train_fs, y_train) #we do it with feature selection
ExtraTreesClassifier(random_state=0)
print("Testing set score with feature selection")
print(clf.score(x_test_fs,y_test))

Testing set score with feature selection
0.7947860962566845


#Final Metrics

In [84]:
from sklearn.metrics import f1_score
print("f1 score:")
print(f1_score(y_test, clf.predict(x_test_fs), average='micro'))

from sklearn.metrics import cohen_kappa_score
kappa = cohen_kappa_score(clf.predict(x_test_fs),y_test,weights=None)
print("kappa scoreL")
print(kappa)

f1 score:
0.7947860962566845
kappa score
0.6616239944064723
